# Roboporter - 2016

## RC Tests

This module is set to be used to test the Robot as a remote controlled car. The operation uses a server/client architecture with the server running on a Raspberry Pi 3 and the a PC command terminal connecting to it through python sockets as a 
client. (note to self -  this is test 4 in base directory) 

### Server Code

Import the required python modules

In [2]:
import socket
import serial
#import fcntl #linux specific (keep note)
import struct
import threading
import time
import numpy

Define Global variables

In [ ]:
global lastCommand
lastCommand= ""

In [ ]:
global serialConnected
serialConnected = False
global motorConn

In [3]:
dataInput = ""
exitFlag = 0
USAvgDistances = []
porterLocation = numpy.array([0, 0, 0])

In [ ]:
def get_ip_address(ifname):
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    print("Resolving ip address")
    return socket.inet_ntoa(fcntl.ioctl(
        s.fileno(),
        0x8915,  # SIOCGIFADDR
        struct.pack('256s', ifname[:15])
    )[20:24])

In [ ]:
class multiThreadBase (threading.Thread):
    def __init__(self, threadID, name):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name

In [ ]:
class SerialThread (threading.Thread):
    def __init__(self, threadID, name):
        threading.Thread.__init__(self)
        self.threadID = threadID

    def run(self):
        print "Starting " + self.name
        send_serial_data(self.name)
        print "Exiting " + self.name


In [ ]:
class usDataThread (multiThreadBase):
    def __init__(self, threadID, name):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.rawUSdata = []
        self.inputBuf = ""

    def run(self):
        print "Starting " + self.name
        self.getUSvector()
        self.mAverage(5)
        print "Exiting " + self.name

    def getUSvector(self):
        pass

    def mAverage(self, n):
        i = 0
        for i in [0, 6]:
            USAvgDistances[i] = USAvgDistances[i] + (self.rawUSdata[i] - USAvgDistances[i])/n


The function below is used to update the location of the porter in a parallel fashion when the actuators successfully compelte the current motion

In [ ]:
def send_serial_data(threadName):
    while not exitFlag:
        try:
            print("Instructing to go at " + lastCommand)
            motorConn.write(lastCommand)

            print("Successfully sent...")
            #print ("Motor says - " + motorConn.readline())
        except Exception as e:
            print ("ERROR - " + str(e))
            try:
                print ("Trying to open serial port")
                motorConn.open()
                serialConnected = True
            except Exception as e:
                print("ERROR - Serial port couldn't be opened :( : " + str(e))
            finally:
                print ("No serial Comms... Looping back to listening mode")
        time.sleep(1.5)

In [ ]:
#set the server address and port
print("Setting up sockets...")
HOST =  get_ip_address('wlan0') #socket.gethostbyname(socket.gethostname()) #socket.gethostname()
PORT = 5002

#create a socket to establish a server
print("Binding the socket...")
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((HOST, PORT))

In [ ]:
#listen to incoming connections on PORT
print 'Socket opened at ', HOST, 'listening to port ', PORT, '\n'
s.listen(1)

In [ ]:
#setup serial connection to motor controller
print("Trying to connect to serial devices")
try:
    motorConn = serial.Serial('/dev/ttyACM0', 19200) #check this
    serialConnected = True
    print ('Connected to serial port /dev/ttyACM0')
except Exception as e:
    print ('Unable to establish serial comms to port /dev/ttyACM0')

In [ ]:
serialThread = SerialThread(1, "serial com thread")
serialThread.start()

In [ ]:
while True:
    #for each connection received create a tunnel to the client
    print ("ready for a new client to connect...")
    clientConnection, address = s.accept()
    print 'Connected by', address

    #send welcome message
    print ("Sending welcome message...")
    clientConnection.send('Connection ack')
    dataInput = clientConnection.recv(1024)
    print ("Client says - " + dataInput)
    dataInput = ""

    while True:
        dataInput = clientConnection.recv(1024)
        if dataInput == "e":
            break
        elif dataInput == "q":
            break
        else:
            print ("Client says - " + dataInput)
            lastCommand = dataInput
            # if dataInput[0] == "#":
            #     print ("Valid Command")
            #     lastCommand = dataInput[1:len(dataInput)]
            # else:
            #     print ("Invalid Command")

        print ("")
    #shut down the server
    clientConnection.close()
    print ("client at " + str(address) + " closed the connection ")
    if dataInput == "q":
        print ("Shutting down the server at " + HOST + "...")
        exitFlag = 1
        s.close()
        break

### Client Code

In [ ]:
import socket
import sys
from msvcrt import getch

In [ ]:
#set the server address and port
HOST = raw_input("Please enter the server address: ") 
PORT = 5002

In [ ]:
#create a socket to connect to the server
s = socket.socket()

#connect to the server at HOST through PORT
print 'Trying to connect to ', HOST, 'at port ', PORT
s.connect((HOST, PORT))

#if connected (add error checking)

#recieve welcome message
print (("Server says - " + s.recv(1024)))

#initialise user input buffer and notify the server (for debugging)
usInput = ""
s.send("I am about to send data")

In [ ]:
#while the user doesnt stop communication ...
while True:
    usInput = raw_input("input control command - ")

    if usInput == "exit": #if ESC break the loop
        print ("closing the connection to " + HOST + "...")
        s.send("e")
        break
    if usInput == "shutdown":
        print ("Instructing the server at " + HOST + " to shut down...")
        s.send("q")
        print ("closing the connection to " + HOST + "...")
        break
    inputBuf = s.recv(1024)
    #send data to server
    s.send(usInput)

#close connection once transmissions are done.
s.close()